In [4]:
!pip install federpy deepface hnswlib numpy opencv-python IPython ipykernel IProgress tqdm tensorflow

Looking in links: http://pypi.org, http://files.pythonhosted.org, http://developer.download.nvidia.com, http://download.pytorch.org, http://dl.fbaipublicfiles.com

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install ipywidgets

In [1]:
import os
import cv2
import pickle
import hnswlib
import numpy as np
from glob import glob
import tensorflow as tf
from deepface import DeepFace
from tqdm.notebook import tqdm
from federpy.federpy import FederPy
from sklearn.preprocessing import normalize

2023-10-13 09:43:32.654579: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Hide GPU from visible devices
tf.config.set_visible_devices([], 'GPU')

2023-10-13 09:43:34.887811: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-13 09:43:34.888009: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-13 09:43:34.909255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
# patterns of image paths
to_insert = glob('./faces/*/*.JPEG')
to_insert.extend(glob('./faces/*/*.jpg'))
to_insert.extend(glob('./faces/*/*.png'))
to_insert.extend(glob('./faces/*/*.jpeg'))

In [4]:
embedding_objs = DeepFace.represent(img_path = "probe.png", model_name="Facenet512", enforce_detection=False)[0]['embedding']


In [6]:
# Declaring index
graph = hnswlib.Index(space = 'cosine', dim = 512)
# Initializing index - the maximum number of elements should be known beforehand
graph.init_index(max_elements = 100000, ef_construction = 100, M = 32)

In [7]:
for id, data in enumerate(tqdm(to_insert)):
  vec = DeepFace.represent(img_path = data, model_name="Facenet512", enforce_detection=False)[0]['embedding']
  vec = normalize([vec])[0]
  graph.add_items(vec, id)

  0%|          | 0/2375 [00:00<?, ?it/s]

In [8]:
graph.save_index("hnswSource")

In [5]:
# init federpy
hnswViewParams = {
    "width": 1000,
    "height": 700,
    "mediaType": "image",
    "mediaUrls": to_insert
}
federPy_hnsw = FederPy("hnswSource", 'hnswlib', **hnswViewParams)
federPy_hnsw.overview(True)

In [6]:

federPy_hnsw.searchRandTestVec()

In [7]:
vec = DeepFace.represent(img_path = "probe.png", model_name="Facenet512", enforce_detection=False)[0]['embedding']
vec = normalize([vec])[0]
federPy_hnsw.setSearchParams({"k": 1}).searchByVec(vec)

In [ ]:
federPy_hnsw = FederPy("hnswSource", 'hnswlib')
html_content = federPy_hnsw.overview(False)
with open("example.html", "w") as file:
    # Write the HTML content to the file
    file.write(html_content)